In [1]:
import random
import pandas as pd

# Define a function to create synthetic user profiles
def create_user_profiles(num_users):
    user_profiles = []
    for user_id in range(1, num_users + 1):
        user_profiles.append({
            'user_id': user_id,
            'age': random.randint(18, 70),
            'gender': random.choice(['M', 'F']),
            'preferred_location': random.choice(['beach', 'city center', 'mountain']),
            'preferred_hotel_type': random.choice(['luxury', 'budget', 'boutique']),
            'avg_rating_given': random.uniform(3.0, 5.0)
        })
    return pd.DataFrame(user_profiles)

# Create 100 synthetic user profiles
user_profiles = create_user_profiles(100)
print(user_profiles.head())


   user_id  age gender preferred_location preferred_hotel_type  \
0        1   34      F              beach               luxury   
1        2   55      F        city center               budget   
2        3   20      M        city center               budget   
3        4   53      M              beach               budget   
4        5   64      F              beach             boutique   

   avg_rating_given  
0          4.000708  
1          4.890188  
2          4.444119  
3          4.742092  
4          4.733793  


In [2]:
# Define a function to simulate user interactions with hotels
def simulate_interactions(user_profiles, hotel_data, num_interactions):
    interactions = []
    for _ in range(num_interactions):
        user = user_profiles.sample(1).iloc[0]
        hotel = hotel_data.sample(1).iloc[0]
        interaction = {
            'user_id': user['user_id'],
            'hotel_id': hotel['hotel_id'],
            'viewed': random.choice([0, 1]),
            'clicked': random.choice([0, 1]),
            'rating': random.uniform(1.0, 5.0) if random.choice([0, 1]) else None,
            'booked': random.choice([0, 1])
        }
        interactions.append(interaction)
    return pd.DataFrame(interactions)

# Example hotel data (replace with your scraped data)
hotel_data = pd.DataFrame({
    'hotel_id': range(1, 21),
    'name': [f'Hotel {i}' for i in range(1, 21)],
    'location': random.choices(['beach', 'city center', 'mountain'], k=20),
    'price_per_night': random.choices(range(50, 500, 50), k=20),
    'star_rating': random.choices(range(1, 6), k=20),
    'amenities': [random.choices(['Wi-Fi', 'Breakfast included', 'Swimming pool'], k=3) for _ in range(20)]
})

# Simulate 1000 user interactions
interactions = simulate_interactions(user_profiles, hotel_data, 1000)
print(interactions.head())


   user_id  hotel_id  viewed  clicked    rating  booked
0       66        17       0        0  2.099695       1
1       72         7       0        0       NaN       0
2       17         4       0        0  3.902601       0
3        7         7       0        0       NaN       1
4       45        10       1        1       NaN       0


In [3]:
# Define a function to simulate user interactions with hotels
def simulate_interactions(user_profiles, hotel_data, num_interactions):
    interactions = []
    for _ in range(num_interactions):
        user = user_profiles.sample(1).iloc[0]
        hotel = hotel_data.sample(1).iloc[0]
        interaction = {
            'user_id': user['user_id'],
            'hotel_id': hotel['hotel_id'],
            'viewed': random.choice([0, 1]),
            'clicked': random.choice([0, 1]),
            'rating': random.uniform(1.0, 5.0) if random.choice([0, 1]) else None,
            'booked': random.choice([0, 1])
        }
        interactions.append(interaction)
    return pd.DataFrame(interactions)

# Example hotel data (replace with your scraped data)
hotel_data = pd.DataFrame({
    'hotel_id': range(1, 21),
    'name': [f'Hotel {i}' for i in range(1, 21)],
    'location': random.choices(['beach', 'city center', 'mountain'], k=20),
    'price_per_night': random.choices(range(50, 500, 50), k=20),
    'star_rating': random.choices(range(1, 6), k=20),
    'amenities': [random.choices(['Wi-Fi', 'Breakfast included', 'Swimming pool'], k=3) for _ in range(20)]
})

# Simulate 1000 user interactions
interactions = simulate_interactions(user_profiles, hotel_data, 1000)
print(interactions.head())


KeyboardInterrupt: 

In [ ]:
# Combine user profiles and interactions with hotel data
combined_data = interactions.merge(user_profiles, on='user_id').merge(hotel_data, on='hotel_id')
print(combined_data.head())

# Preprocess the combined data (handle missing values, encode features, etc.)
combined_data['rating'] = combined_data['rating'].fillna(combined_data['rating'].mean())
# Further preprocessing steps as needed


   user_id  hotel_id  viewed  clicked    rating  booked  age gender  \
0       64         2       0        0       NaN       1   36      M   
1       64         2       0        0  3.778714       0   36      M   
2       67         2       1        1       NaN       1   36      F   
3       73         2       0        0       NaN       0   58      F   
4       73         2       1        1  1.182294       1   58      F   

  preferred_location preferred_hotel_type  avg_rating_given     name  \
0        city center               budget          3.831872  Hotel 2   
1        city center               budget          3.831872  Hotel 2   
2        city center               budget          4.003608  Hotel 2   
3        city center             boutique          4.710647  Hotel 2   
4        city center             boutique          4.710647  Hotel 2   

      location  price_per_night  star_rating  \
0  city center              100            3   
1  city center              100            3

In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

# Prepare data for Surprise library
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(combined_data[['user_id', 'hotel_id', 'rating']], reader)

# Use Singular Value Decomposition (SVD) for collaborative filtering
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Train the model on the entire dataset
trainset = data.build_full_trainset()
svd.fit(trainset)

# Predict ratings for a user (user_id = 1) for a hotel (hotel_id = 1)
prediction = svd.predict(uid=1, iid=1)
print(prediction)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9709  0.8063  0.8192  0.8396  0.9233  0.8719  0.0641  
MAE (testset)     0.6932  0.5565  0.5802  0.5713  0.6624  0.6127  0.0545  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    
user: 1          item: 1          r_ui = None   est = 3.03   {'was_impossible': False}


In [ ]:
import random
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

# Define a function to create synthetic user profiles
def create_user_profiles(num_users):
    user_profiles = []
    for user_id in range(1, num_users + 1):
        user_profiles.append({
            'user_id': user_id,
            'age': random.randint(18, 70),
            'gender': random.choice(['M', 'F']),
            'preferred_location': random.choice(['beach', 'city center', 'mountain']),
            'preferred_hotel_type': random.choice(['luxury', 'budget', 'boutique']),
            'avg_rating_given': random.uniform(3.0, 5.0)
        })
    return pd.DataFrame(user_profiles)

# Create 100 synthetic user profiles
user_profiles = create_user_profiles(100)

# Example hotel data (replace with your scraped data)
hotel_data = pd.DataFrame({
    'hotel_id': range(1, 21),
    'name': [f'Hotel {i}' for i in range(1, 21)],
    'location': random.choices(['beach', 'city center', 'mountain'], k=20),
    'price_per_night': random.choices(range(50, 500, 50), k=20),
    'star_rating': random.choices(range(1, 6), k=20),
    'amenities': [random.choices(['Wi-Fi', 'Breakfast included', 'Swimming pool'], k=3) for _ in range(20)]
})

# Define a function to simulate user interactions with hotels
def simulate_interactions(user_profiles, hotel_data, num_interactions):
    interactions = []
    for _ in range(num_interactions):
        user = user_profiles.sample(1).iloc[0]
        hotel = hotel_data.sample(1).iloc[0]
        interaction = {
            'user_id': user['user_id'],
            'hotel_id': hotel['hotel_id'],
            'viewed': random.choice([0, 1]),
            'clicked': random.choice([0, 1]),
            'rating': random.uniform(1.0, 5.0) if random.choice([0, 1]) else None,
            'booked': random.choice([0, 1])
        }
        interactions.append(interaction)
    return pd.DataFrame(interactions)

# Simulate 1000 user interactions
interactions = simulate_interactions(user_profiles, hotel_data, 1000)

# Combine user profiles and interactions with hotel data
combined_data = interactions.merge(user_profiles, on='user_id').merge(hotel_data, on='hotel_id')

# Preprocess the combined data (handle missing values, encode features, etc.)
combined_data['rating'] = combined_data['rating'].fillna(combined_data['rating'].mean())

# Prepare data for Surprise library
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(combined_data[['user_id', 'hotel_id', 'rating']], reader)

# Use Singular Value Decomposition (SVD) for collaborative filtering
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Train the model on the entire dataset
trainset = data.build_full_trainset()
svd.fit(trainset)

# Predict ratings for a user (user_id = 1) for a hotel (hotel_id = 1)
prediction = svd.predict(uid=1, iid=1)
print(prediction)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8383  0.7986  0.9264  0.8932  0.8431  0.8599  0.0448  
MAE (testset)     0.5929  0.5445  0.6709  0.6399  0.5912  0.6079  0.0436  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    
user: 1          item: 1          r_ui = None   est = 3.09   {'was_impossible': False}


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Function to scrape hotel reviews from TripAdvisor
def scrape_tripadvisor_hotel_reviews(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    # Retry mechanism
    session = requests.Session()
    retry = Retry(
        total=5,
        read=5,
        connect=5,
        backoff_factor=0.3,
        status_forcelist=(500, 502, 504)
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    try:
        page = session.get(url, headers=headers, timeout=30)  # Increase the timeout duration
        page.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if the request fails
    
    soup = BeautifulSoup(page.content, 'html.parser')

    reviews = []
    
    # Attempt to find the hotel name
    hotel_name_tag = soup.find('h1')
    hotel_name = hotel_name_tag.text if hotel_name_tag else 'Unknown Hotel'

    # Attempt to find the review containers
    review_containers = soup.find_all('div', class_='review-container')
    
    if not review_containers:
        print(f"No reviews found on page: {url}")
    
    for review in review_containers:
        try:
            user_id = review.find('div', class_='info_text pointer_cursor').text.strip()
            rating_class = review.find('span', class_='ui_bubble_rating')['class']
            rating = int(rating_class[1].split('_')[1]) / 10
            review_title = review.find('a', class_='title').text.strip()
            review_text = review.find('q', class_='IRsGHoPm').text.strip()

            review_data = {
                'hotel_name': hotel_name,
                'user_id': user_id,
                'rating': rating,
                'review_title': review_title,
                'review_text': review_text,
            }
            reviews.append(review_data)
        except AttributeError as e:
            print(f"Error parsing review: {e}")
            continue

    return pd.DataFrame(reviews)

# Example URL of a TripAdvisor hotel page
url = 'https://www.tripadvisor.com/Hotel_Review-g187791-d203127-Reviews-Hotel_Delle_Nazioni-Rome_Lazio.html'
hotel_reviews = scrape_tripadvisor_hotel_reviews(url)
print(hotel_reviews)


Request failed: HTTPSConnectionPool(host='www.tripadvisor.com', port=443): Max retries exceeded with url: /Hotel_Review-g187791-d203127-Reviews-Hotel_Delle_Nazioni-Rome_Lazio.html (Caused by ReadTimeoutError("HTTPSConnectionPool(host='www.tripadvisor.com', port=443): Read timed out. (read timeout=30)"))
Empty DataFrame
Columns: []
Index: []


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv

# Initialize the WebDriver (Ensure you have the correct driver for your browser installed)
driver = webdriver.Chrome(executable_path='/path/to/chromedriver')

def get_review_data(url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

    reviews = []
    review_elements = driver.find_elements(By.CLASS_NAME, 'review-container')

    for review_element in review_elements:
        review = {}
        try:
            review['title'] = review_element.find_element(By.CLASS_NAME, 'noQuotes').text
        except:
            review['title'] = 'N/A'
        
        try:
            review['rating'] = review_element.find_element(By.CSS_SELECTOR, 'span.ui_bubble_rating').get_attribute('class').split('_')[-1]
        except:
            review['rating'] = 'N/A'
        
        try:
            review['text'] = review_element.find_element(By.CLASS_NAME, 'IRsGHoPm').text
        except:
            review['text'] = 'N/A'
        
        try:
            user_info = review_element.find_element(By.CLASS_NAME, 'info_text')
            review['username'] = user_info.find_element(By.CLASS_NAME, 'username').text
            review['location'] = user_info.find_element(By.CLASS_NAME, 'userLoc').text
            review['contributions'] = user_info.find_element(By.CLASS_NAME, 'badgetext').text
        except:
            review['username'] = 'N/A'
            review['location'] = 'N/A'
            review['contributions'] = 'N/A'

        reviews.append(review)
    return reviews

def save_to_csv(data, filename):
    keys = data[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

# Main function to scrape review data
def main():
    url = 'https://www.tripadvisor.com/Hotel_Review-g60763-d93603-Reviews-Hotel_Pennsylvania-New_York_City_New_York.html'  # Update this URL pattern
    all_reviews = []

    for page in range(0, 10):  # Adjust range and step as needed for more pages
        page_url = f"{url}?page={page+1}"
        reviews = get_review_data(page_url)
        all_reviews.extend(reviews)
        time.sleep(2)  # Sleep to avoid being blocked

    # Save all reviews data
    save_to_csv(all_reviews, 'reviews_data.csv')
    driver.quit()

if __name__ == '__main__':
    main()


TypeError: __init__() got an unexpected keyword argument 'executable_path'

In [6]:
import random
import pandas as pd

# Provided hotel data
hotels_data = [
    {"hotel_id": 1, "hotel_name": "A.C.C Design Hotel", "description": "A modern hotel with stylish decor.", "city": "Gwangju", "country": "South Korea", "address": "226-11, Geumnam-ro, Dong-gu, Gwangju 61482 South Korea", "price": None, "rating": 4.0},
    {"hotel_id": 2, "hotel_name": "Dasomchae", "description": "A cozy hotel offering traditional Korean hospitality.", "city": "Gwangju", "country": "South Korea", "address": "27, Naesang-ro 51beon-gil, Gwangsan-gu, Gwangju 62431 South Korea", "price": None, "rating": 4.5},
    {"hotel_id": 3, "hotel_name": "Brown Dot Hotel", "description": "A comfortable hotel in a convenient location.", "city": "Gwangju", "country": "South Korea", "address": "5-8, Sangmujungang-ro 38beon-gil, Seo-gu, Gwangju 61963 South Korea", "price": None, "rating": 3.5},
    {"hotel_id": 4, "hotel_name": "Empire Tourist Hotel", "description": "An affordable hotel for budget-conscious travelers.", "city": "Gwangju", "country": "South Korea", "address": "25, Imbangul-daero 800beon-gil, Gwangsan-gu, Gwangju 62277 South Korea", "price": None, "rating": 3.0},
    {"hotel_id": 5, "hotel_name": "CS Hotel", "description": "A hotel known for its excellent service.", "city": "Gwangju", "country": "South Korea", "address": "128, Sangmupyeonghwa-ro, Seo-gu, Gwangju 61964 South Korea", "price": None, "rating": 3.5},
    # Add all other hotel data here as per your list...
]

# Convert hotels data to DataFrame
hotels_df = pd.DataFrame(hotels_data)

# Function to generate random user data
def generate_user_data(num_users):
    user_data = []
    for user_id in range(1, num_users + 1):
        age = random.randint(18, 70)
        gender = random.choice(['M', 'F'])
        preferred_location = random.choice(['beach', 'mountain', 'city center', 'countryside'])
        preferred_hotel_type = random.choice(['luxury', 'budget', 'boutique'])
        
        user_data.append({
            'user_id': user_id,
            'age': age,
            'gender': gender,
            'preferred_location': preferred_location,
            'preferred_hotel_type': preferred_hotel_type
        })
    return user_data

# Function to generate random reviews
def generate_review_data(num_reviews, num_users, num_hotels):
    review_data = []
    for review_id in range(1, num_reviews + 1):
        user_id = random.randint(1, num_users)
        hotel_id = random.randint(1, num_hotels)
        rating = round(random.uniform(1.0, 5.0), 2)
        review_txt = f'Review text for review_{review_id}'
        
        review_data.append({
            'review_id': review_id,
            'user_id': user_id,
            'hotel_id': hotel_id,
            'rating': rating,
            'review_txt': review_txt
        })
    return review_data

# Generate synthetic data
num_users = 100
num_reviews = 300
num_hotels = len(hotels_data)

synthetic_users = generate_user_data(num_users)
synthetic_reviews = generate_review_data(num_reviews, num_users, num_hotels)

# Convert to DataFrames
users_df = pd.DataFrame(synthetic_users)
reviews_df = pd.DataFrame(synthetic_reviews)

# Save to CSV (optional)
users_df.to_csv('synthetic_usereees.csv', index=False)
hotels_df.to_csv('synthetic_hoteleees.csv', index=False)
reviews_df.to_csv('synthetic_revieweees.csv', index=False)

# Display the first few rows of the DataFrames
print("Users DataFrame:")
print(users_df.head())

print("\nHotels DataFrame:")
print(hotels_df.head())

print("\nReviews DataFrame:")
print(reviews_df.head())


Users DataFrame:
   user_id  age gender preferred_location preferred_hotel_type
0        1   40      M           mountain               luxury
1        2   28      F           mountain               budget
2        3   26      F        countryside               luxury
3        4   38      M              beach               budget
4        5   25      F        city center               luxury

Hotels DataFrame:
   hotel_id            hotel_name  \
0         1    A.C.C Design Hotel   
1         2             Dasomchae   
2         3       Brown Dot Hotel   
3         4  Empire Tourist Hotel   
4         5              CS Hotel   

                                         description     city      country  \
0                 A modern hotel with stylish decor.  Gwangju  South Korea   
1  A cozy hotel offering traditional Korean hospi...  Gwangju  South Korea   
2      A comfortable hotel in a convenient location.  Gwangju  South Korea   
3  An affordable hotel for budget-conscious trave..

In [6]:
pip install pymysql

   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ------------------------------------ --- 41.0/45.0 kB 960.0 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 739.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pymysql

# Load data from CSV files
users_df = pd.read_csv('synthetic_users.csv')
reviews_df = pd.read_csv('synthetic_reviews.csv')

# Database connection details
db_host = 'localhost'
db_user = 'root'
db_password = 'root'
db_name = 'recommendation_system'

# Create a connection to the database
connection = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)

try:
    with connection.cursor() as cursor:
        # Create the users table if it doesn't exist
        create_users_table_query = """
        CREATE TABLE IF NOT EXISTS users (
            user_id VARCHAR(255) PRIMARY KEY,
            age INT,
            gender CHAR(1),
            preferred_location VARCHAR(255),
            preferred_hotel_type VARCHAR(255)
        );
        """
        cursor.execute(create_users_table_query)

        # Insert data into the users table
        for _, row in users_df.iterrows():
            insert_user_query = """
            INSERT INTO users (user_id, age, gender, preferred_location, preferred_hotel_type)
            VALUES (%s, %s, %s, %s, %s)
            """
            cursor.execute(insert_user_query, (row['user_id'], row['age'], row['gender'], row['preferred_location'], row['preferred_hotel_type']))
        
        # Create the reviews table if it doesn't exist
        create_reviews_table_query = """
        CREATE TABLE IF NOT EXISTS reviews (
            user_id VARCHAR(255),
            avg_rating_given FLOAT,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        );
        """
        cursor.execute(create_reviews_table_query)

        # Insert data into the reviews table
        for _, row in reviews_df.iterrows():
            insert_review_query = """
            INSERT INTO reviews (user_id, avg_rating_given)
            VALUES (%s, %s)
            """
            cursor.execute(insert_review_query, (row['user_id'], row['avg_rating_given']))

    # Commit the transaction
    connection.commit()

finally:
    # Close the connection
    connection.close()


OperationalError: (1045, "Access denied for user 'root'@'localhost' (using password: YES)")

In [8]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

# Load data from CSV files
users_df = pd.read_csv('users.csv')
hotels_df = pd.read_csv('hotels_data.csv')
reviews_df = pd.read_csv('reviews.csv')


In [9]:
# Prepare the data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(reviews_df[['user_id', 'hotel_id', 'rating']], reader)


In [10]:
# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Use the SVD algorithm
algo = SVD()

# Train the algorithm on the trainset
algo.fit(trainset)

# Test the algorithm on the testset
predictions = algo.test(testset)

# Compute and print RMSE
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse}')


RMSE: 1.1890
RMSE: 1.1890035761929392


In [11]:
# Predict the rating for a specific user and hotel
user_id = 1  # example user_id
hotel_id = 10  # example hotel_id
prediction = algo.predict(user_id, hotel_id)
print(f'Predicted rating for user {user_id} and hotel {hotel_id}: {prediction.est}')


Predicted rating for user 1 and hotel 10: 3.187858311861201


In [12]:
def recommend_hotels(algo, user_id, n=5):
    # Get a list of all hotel_ids
    hotel_ids = reviews_df['hotel_id'].unique()
    
    # Predict ratings for all hotels for the given user
    predictions = [algo.predict(user_id, hotel_id).est for hotel_id in hotel_ids]
    
    # Pair each hotel_id with its predicted rating
    hotel_ratings = list(zip(hotel_ids, predictions))
    
    # Sort the hotels by predicted rating in descending order
    hotel_ratings.sort(key=lambda x: x[1], reverse=True)
    
    # Get the top N recommended hotels
    recommended_hotels = hotel_ratings[:n]
    
    return recommended_hotels

# Get recommendations for a specific user
user_id = 1  # example user_id
recommendations = recommend_hotels(algo, user_id, n=5)
print(f'Top 5 recommended hotels for user {user_id}: {recommendations}')


Top 5 recommended hotels for user 1: [(1, 3.443091834124955), (5, 3.382604771542623), (3, 3.2211517841606536), (2, 3.1743898571870197), (4, 3.1007577726534046)]


In [15]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

# Load data from CSV files
users_df = pd.read_csv('synthetic_users.csv')
hotels_df = pd.read_csv('synthetic_hotels.csv')
reviews_df = pd.read_csv('synthetic_reviews.csv')

# Prepare the data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(reviews_df[['user_id', 'hotel_id', 'rating']], reader)

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Use the SVD algorithm
algo = SVD()

# Train the algorithm on the trainset
algo.fit(trainset)

# Test the algorithm on the testset
predictions = algo.test(testset)

# Compute and print RMSE
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse}')

# Predict the rating for a specific user and hotel
user_id = 1  # example user_id
hotel_id = 10  # example hotel_id
prediction = algo.predict(user_id, hotel_id)
print(f'Predicted rating for user {user_id} and hotel {hotel_id}: {prediction.est}')

# Recommend Hotels for a User
def recommend_hotels(algo, user_id, n=5):
    # Check if the user_id exists in the training set
    try:
        # Check if the user exists in the training data's raw to inner mapping
        trainset.to_inner_uid(user_id)
        # User exists in training data, use collaborative filtering
        hotel_ids = reviews_df['hotel_id'].unique()
        predictions = [algo.predict(user_id, hotel_id).est for hotel_id in hotel_ids]
        hotel_ratings = list(zip(hotel_ids, predictions))
        hotel_ratings.sort(key=lambda x: x[1], reverse=True)
        recommended_hotels = hotel_ratings[:n]
    except ValueError:
        # New user, use content-based or popular hotels recommendation
        recommended_hotels = recommend_popular_hotels(n)
    return recommended_hotels

# Popular Hotels Recommendation
def recommend_popular_hotels(n=5):
    popular_hotels = reviews_df.groupby('hotel_id').agg({'rating': 'mean', 'review_id': 'count'}).reset_index()
    popular_hotels = popular_hotels.sort_values(by=['rating', 'review_id'], ascending=False)
    recommended_hotels = popular_hotels.head(n)['hotel_id'].tolist()
    return recommended_hotels

# Content-Based Recommendation
def recommend_hotels_content_based(user_profile, n=5):
    filtered_hotels = hotels_df[
        (hotels_df['description'].str.contains(user_profile['preferred_hotel_type'], case=False, na=False)) |
        (hotels_df['description'].str.contains(user_profile['preferred_location'], case=False, na=False))
    ]
    filtered_hotels = filtered_hotels.sort_values(by='rating', ascending=False)
    recommended_hotels = filtered_hotels.head(n)['hotel_id'].tolist()
    return recommended_hotels

# Example user profile for content-based recommendation
new_user_profile = {'age': 30, 'gender': 'M', 'preferred_location': 'beach', 'preferred_hotel_type': 'luxury'}

# Get recommendations for a specific user (existing user example)
existing_user_id = 1
existing_recommendations = recommend_hotels(algo, existing_user_id, n=5)
print(f'Top 5 recommended hotels for existing user {existing_user_id}: {existing_recommendations}')

# Get recommendations for a specific user (new user example)
new_user_id = 101  # New user (not in training data)
new_recommendations = recommend_hotels(algo, new_user_id, n=5)
print(f'Top 5 recommended hotels for new user {new_user_id}: {new_recommendations}')

# Content-based recommendations for new user profile
content_based_recommendations = recommend_hotels_content_based(new_user_profile, n=5)
print(f'Content-based recommendations: {content_based_recommendations}')


RMSE: 1.1831
RMSE: 1.1831275639773278
Predicted rating for user 1 and hotel 10: 3.019083333333333
Top 5 recommended hotels for existing user 1: ['h_25', 'h_33', 'h_26', 'h_29', 'h_49']
Top 5 recommended hotels for new user 101: ['h_25', 'h_33', 'h_26', 'h_29', 'h_49']
Content-based recommendations: []


In [16]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import joblib
import pandas as pd

# Load data
reviews_df = pd.read_csv('synthetic_reviews.csv')

# Prepare the data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(reviews_df[['user_id', 'hotel_id', 'rating']], reader)

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Use the SVD algorithm
algo = SVD()

# Train the algorithm on the trainset
algo.fit(trainset)

# Save the trained model
joblib.dump(algo, 'svd_model.pkl')



['svd_model.pkl']

In [17]:
pip install requests beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests

url = 'https://www.tripadvisor.com/Hotels'
response = requests.get(url)
html_content = response.text


In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_content, 'html.parser')


In [3]:
images = soup.find_all('img', {'class': 'hotel_image_class'})  # Replace 'hotel_image_class' with the actual class name of the images
image_urls = [img['src'] for img in images if 'src' in img.attrs]


In [4]:
import os

os.makedirs('hotel_images', exist_ok=True)

for i, url in enumerate(image_urls):
    image_response = requests.get(url)
    with open(f'hotel_images/hotel_image_{i}.jpg', 'wb') as file:
        file.write(image_response.content)


In [5]:
import requests
from bs4 import BeautifulSoup
import os

def fetch_hotel_images(url):
    # Step 1: Send a request to the website
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return []

    # Step 2: Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Step 3: Find and extract image URLs
    images = soup.find_all('img', {'class': 'hotel_image_class'})  # Adjust the class name as needed
    image_urls = [img['src'] for img in images if 'src' in img.attrs]

    return image_urls

def download_images(image_urls):
    os.makedirs('hotel_images', exist_ok=True)

    for i, url in enumerate(image_urls):
        try:
            image_response = requests.get(url)
            with open(f'hotel_images/hotel_image_{i}.jpg', 'wb') as file:
                file.write(image_response.content)
        except Exception as e:
            print(f"Failed to download image {url}: {e}")

if __name__ == "__main__":
    url = 'https://www.tripadvisor.com/Hotels'
    image_urls = fetch_hotel_images(url)
    if image_urls:
        download_images(image_urls)
    else:
        print("No images found.")


In [1]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import os
import time

def fetch_hotel_images(url):
    # Set up Selenium WebDriver
    driver = webdriver.Chrome()  # Ensure you have the ChromeDriver installed and in your PATH
    driver.get(url)

    # Allow time for the page to load
    time.sleep(5)

    # Find image elements
    images = driver.find_elements(By.TAG_NAME, 'img')

    # Extract image URLs
    image_urls = [img.get_attribute('src') for img in images if img.get_attribute('src')]

    # Print the number of images found
    print(f"Number of images found: {len(image_urls)}")

    # Close the browser
    driver.quit()

    return image_urls

def download_images(image_urls):
    os.makedirs('hotel_images', exist_ok=True)

    for i, url in enumerate(image_urls):
        try:
            image_response = requests.get(url)
            with open(f'hotel_images/hotel_image_{i}.jpg', 'wb') as file:
                file.write(image_response.content)
        except Exception as e:
            print(f"Failed to download image {url}: {e}")

if __name__ == "__main__":
    url = 'https://www.tripadvisor.com/Hotels'
    image_urls = fetch_hotel_images(url)
    if image_urls:
        download_images(image_urls)
    else:
        print("No images found.")


In [1]:
import mysql.connector
import pandas as pd

# Database connection configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root',
    'database': 'recommendation_system'
}

# Connect to the database
conn = mysql.connector.connect(**db_config)

# Define the query to select the data from the table
query = "SELECT * FROM hotels"

# Use pandas to read the data from the database
df = pd.read_sql(query, conn)

# Specify the path where you want to save the CSV file
csv_file_path = 'output.csv'

# Export the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

# Close the database connection
conn.close()

print(f"Data exported to {csv_file_path}")


SyntaxError: invalid syntax (3528830763.py, line 1)